In [1]:
import requests
from bs4 import BeautifulSoup
import dryscrape
from time import sleep

In [2]:
newspaper = {}
newspaper["searchUrl"] = "http://www.milenio.com/buscador/"
newspaper["searchParamType"] = "params"
newspaper["searchParam"] = "text"
newspaper["paginationFirstParam"] = "True"
newspaper["paginationStrategy"] = "Refresh"
newspaper["paginationParams"] = {
    "refreshButton": {'linkLocation':'a#busqueda-vermas', "limit":"remove"}
    }
newspaper["visitType"] = "JavascriptEnabled"
newspaper["searchLinksBody"] = "div.results"
newspaper["article"] = {"locationBody":"div.mce-body", "locationTitle":"h1.pg-bkn-headline"}

In [3]:
searchData = {}
searchData["query"] = "Dragon Ball"

In [4]:
def buildBaseUrl(newspaper, searchData):
    if newspaper["searchParamType"] =="endpoint":
        url = newspaper["searchUrl"] + searchData["query"]
    else:
        #Case if searchParamType == get param
        url = newspaper["searchUrl"] +"?" + newspaper["searchParam"] + "=" + searchData["query"] 
    return url

In [5]:
def buildPagination(newspaper, baseUrl, iterationParametersValues, iterationParameter):
    if newspaper["paginationStrategy"] == "GET":
        url, headers, formData = GETPagination(newspaper, baseUrl, iterationParametersValues, iterationParameter)
        print("Getting: "+ url)
        content = visitPage(url,newspaper) #TODO(ricalanis). Include headers and form data.
    return content

In [6]:
def GETPagination(newspaper, baseUrl, iterationParametersValues, iterationParameter):
    url = baseUrl
    firstParam = bool(newspaper["paginationFirstParam"])
    for numParam, param in enumerate(newspaper["paginationParams"]):
        if numParam == 0:
            if firstParam == "True":
                url = url + "?"
            else:
                url = url + "&"
        else:
            url = url + "&"
        url = url + newspaper["paginationParams"][param]["name"]  + "=" + str(iterationParametersValues[param])
    return url, [], []

In [33]:
def searchPageRefresh(newspaper,searchData):
    searchPage = buildBaseUrl(newspaper,searchData)
    sess = dryscrape.Session(searchPage)
    print("Setting up Virtual Explorer")
    content = maxOutPage(searchPage, newspaper)
    stories = extractPageStories(content, newspaper, searchData)
    len(stories)
    return stories

In [42]:
searchPageRefresh(newspaper,searchData)

Setting up Virtual Explorer



AttributeError: 'NoneType' object has no attribute 'findAll'

In [29]:
def maxOutPage(searchPage, newspaper):
    content = visitPage(searchPage, newspaper)
    global sess
    sess = dryscrape.Session(base_url = searchPage)
    button = sess.at_xpath(domToXpath(newspaper["paginationParams"]["refreshButton"]["linkLocation"]))
    "Refreshing Page"
    while button is not None:
        "Refreshing Page"
        button.click()
    "Finished Refreshing, Extracting Page"
    content = BeautifulSoup(sess.body())
    return content

In [9]:
def searchPages(newspaper,searchData):
    searchPage = buildBaseUrl(newspaper,searchData)
    content = visitPage(searchPage, newspaper)
    maxValuesParams = extractMaxParamValue(content, newspaper)
    stories = []
    iterationMax, iterationParametersValues, iterationParameter = getIterationProcess(maxValuesParams, newspaper)
    while iterationParametersValues[iterationParameter] <= maxValuesParams[iterationParameter]:
        print(str(iterationParametersValues[iterationParameter]) + "/" + str(iterationMax))
        content =buildPagination(newspaper, searchPage, iterationParametersValues, iterationParameter)
        stories = stories + extractPageStories(content, newspaper, searchData)
        iterationParametersValues = incrementIterationParameters(iterationParametersValues, newspaper)
        sleep(0.5)
        print("Current story size:" + str(len(stories)))
    return stories

In [10]:
def incrementIterationParameters(iterationParametersValues, newspaper):
    for key in iterationParametersValues:
        print(newspaper["paginationParams"][key]["sumFactor"])
        iterationParametersValues[key] = iterationParametersValues[key] + newspaper["paginationParams"][key]["sumFactor"]
    return iterationParametersValues

In [11]:
def getIterationProcess(maxValuesParams, newspaper):
    iterationMax = 2
    sumFactor = 1
    if "pageNumber" in maxValuesParams:
        #TODO(ricalanis): Abstract this.
        iterationMax = maxValuesParams["pageNumber"]
        iterationParameter = "pageNumber"
    else:
        if "startingPoint" in maxValuesParams:
            iterationMax = maxValuesParams["startingPoint"]
            iterationParameter = "startingPoint"
    iterationParametersValues = {}
    for key in maxValuesParams:
        iterationParametersValues[key] = 1
    return iterationMax, iterationParametersValues, iterationParameter

In [12]:
#non iterative version:
def searchPage(newspaper,searchData):
    searchPage = buildBaseUrl(newspaper,searchData)
    content = visitPage(searchPage, newspaper)
    maxValuesParams = extractMaxParamValue(content, newspaper)
    currentPosition = 1
    stories = []
    stories = stories + extractPageStories(content, newspaper, searchData)
    return stories

In [31]:
def visitPage(page,newspaper):
    if newspaper["visitType"] == "pureRequest":
        content = requests.get(page).content
    elif newspaper["visitType"] == "JavascriptEnabled":
        if not sess: 
            global sess 
            sess = dryscrape.Session(page)
        content = sess.body()
    content = BeautifulSoup(content)
    return content
        

<ipython-input-31-bcb8d664d7a0>:6: SyntaxWarning: name 'sess' is used prior to global declaration
  global sess


In [28]:
def extractMaxParamValue(content,newspaper):
    totalParamValues = {}
    for param in newspaper["paginationParams"]:
        locationText = textExtract(content, newspaper["paginationParams"][param]["totalLocationName"])
        value = int(eval(newspaper["paginationParams"][param]["totalLocationOps"].replace("locationText",'"""'+locationText+'"""')))
        totalParamValues[param] = value
    return totalParamValues
        

In [15]:
def domTranslate(expression):
    if "#" in expression:
        tag, tag_id = expression.split("#")
        return tag, "id", tag_id
    elif "." in expression:
        tag, tag_class = expression.split(".")
        return tag, "class", tag_class
    else:
        raise "No Id or Class symbol found"
    return None

In [41]:
def linkAppend(content, newspaper):
    tag, tagType, tagName = domTranslate(newspaper["searchLinksBody"])
    print(content)
    return list(set([link["href"] for link in content.find(tag,{tagType:tagName}).findAll("a")]))

In [17]:
def textExtract(content, domObject):
    tag, tagType, tagValue = domTranslate(domObject)
    try:
        content.find(tag, {tagType:tagValue})
        locationText = content.find(tag, {tagType:tagValue}).text
    except: 
        locationText = "N/A"
    return locationText

In [37]:
def extractPageStories(content, newspaper,searchData):
    stories = []
    links = linkAppend(content, newspaper)
    for link in links:
        story = {}
        content = visitPage(link,newspaper)
        story["link"] = link
        story["query"] = searchData["query"]
        for articleSection in newspaper["article"]:
            domSection = newspaper["article"][articleSection]
            story[articleSection] = textExtract(content, domSection)
        stories.append(story)
    return stories
        

In [19]:
sess = dryscrape.Session()
sess.visit("http://www.milenio.com/buscador/?text=pokemon")
sess.render("example.png")

In [20]:
button = sess.at_xpath("//a[@id='busqueda-vermas']")
print(button)

<Node #/html/body/div[@id='doc']/div[@id='container']/div[2]/div[@id='main']/div/div[2]/div[1]/div/div/div[3]/p/a[@id='busqueda-vermas']>


In [21]:
sess.render("example2.png")

In [22]:
def domToXpath(expression):
    if "#" in expression:
        tag, tag_id = expression.split("#")
        return "//"+tag+ "[@id='"+ tag_id+"']"
    elif "." in expression:
        tag, tag_class = expression.split(".")
        return "//"+tag+ "[@class='"+ tag_id+"']"
    else:
        raise "No Id or Class symbol found"
    return None

In [23]:
domToXpath(newspaper["paginationParams"]["refreshButton"]["linkLocation"])

"//a[@id='busqueda-vermas']"

In [ ]:
sess.

In [35]:
def linkAppend(content, newspaper):
    tag, tagType, tagName = domTranslate(newspaper["searchLinksBody"])
    return list(set([link["href"] for link in content.find(tag,{tagType:tagName}).findAll("a")]))